In [1]:
import sys
sys.path.append('../src')

from data_utils import preprocess, make_train_valid_dfs, build_loaders
from config import default_config
from transformers import DistilBertTokenizer
from config import default_config

In [2]:
config = default_config
config

{'raw_file_path': '../input/raw/flickr30k/results.csv',
 'clean_file_path': '../input/clean/flickr30k/captions.csv',
 'image_path': '../input/raw/flickr30k/Images',
 'train_size': 0.8,
 'batch_size': 32,
 'num_workers': 4,
 'text_tokenizer': 'distilbert-base-uncased',
 'max_length': 200,
 'image_size': 224}

In [3]:
preprocess(config['raw_file_path'], 1)

Created clean csv file ../input/clean/flickr30k/captions.csv


,image,caption,id
0,1000092795.jpg,Two young guys with shaggy hair look at their ...,0
1,1000092795.jpg,"Two young , White males are outside near many ...",0
2,1000092795.jpg,Two men in green shirts are standing in a yard .,0
3,1000092795.jpg,A man in a blue shirt standing in a garden .,0
4,1000092795.jpg,Two friends enjoy time spent together .,0
...,...,...,...
158910,998845445.jpg,A man in shorts and a Hawaiian shirt leans ove...,31782
158911,998845445.jpg,"A young man hanging over the side of a boat , ...",31782
158912,998845445.jpg,A man is leaning off of the side of a blue and...,31782
158913,998845445.jpg,"A man riding a small boat in a harbor , with f...",31782


In [4]:
train_df, valid_df = make_train_valid_dfs(config["clean_file_path"], 0.8)

In [5]:
tokenizer = DistilBertTokenizer.from_pretrained(config['text_tokenizer'])
train_loader = build_loaders(train_df, tokenizer, mode="train", config=config)

In [6]:
batch = next(iter(train_loader))
batch.keys()

dict_keys(['input_ids', 'attention_mask', 'image', 'caption'])

In [7]:
batch

{'input_ids': tensor([[  101,  1037,  2450,  ...,     0,     0,     0],
         [  101,  2048,  2111,  ...,     0,     0,     0],
         [  101,  1037,  2450,  ...,     0,     0,     0],
         ...,
         [  101,  1037,  2158,  ...,     0,     0,     0],
         [  101,  1037,  2450,  ...,     0,     0,     0],
         [  101,  1996, 21776,  ...,     0,     0,     0]]),
 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         ...,
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0]]),
 'image': tensor([[[[-0.4568, -0.3712, -0.2684,  ..., -1.1760, -0.6794, -0.9020],
           [-0.4226, -0.4226, -0.2856,  ..., -1.2103, -0.9020, -1.0904],
           [-0.5767, -0.4911, -0.4911,  ..., -1.1418, -1.1075, -1.0733],
           ...,
           [ 0.7248,  0.7591,  0.6563,  ...,  0.5707,  0.4851,  0.1939],
           [ 0.7591,  0.7419,  0.8104,  ...,  0.7419,  0